# Automated analysis of the property from section 5.2
This is a simple example of how ultrametric integral cryptanalysis can be automated. For a more complete and performant implementation, we refer to the rest of the code in this repository.

In this example we will perform the same analysis as in section 5.2, but automated. That is, we will analyse a four round property with the dominant trail approximation of Theorem 5.1 with $\Lambda = \emptyset$. We will go through the following steps:
1) Compute the ultrametric integral transition matrix of the PRESENT SBox and the conditions for non-zero correlation trail transition and the corresponding factor of the correlation.
2) Implement conditions for the transition through the key-addition
3) Model the PRESENT round function
4) Model the input set
5) Construct and evaluate the complete model for 4 rounds of PRESENT

In [1]:
import numpy as np
## 1 Ultrametric integral transition matrix

# Computes the ultrametric transition matrix of a vectorial Boolean function
def UT_matrix(f, input_size, output_size):
    def row_transform(M):
        ns = M.shape[0] // 2
        if ns > 0:
            x = row_transform(M[:ns, :])
            y = row_transform(M[ns:, :])
        else:
            return M
        return np.vstack([x+y, y])

    def column_transform(M):
        ns = M.shape[1] // 2
        if ns > 0:
            x = column_transform(M[:, :ns])
            y = column_transform(M[:, ns:])
        else:
            return M
        return np.hstack([x, y-x])

    # Transition matrix over the natural numbers
    res = np.zeros((2**output_size, 2**input_size), dtype='longlong')
    for i in range(2**input_size):
        res[f(i), i] = 1
        
    # transform in place to ultrametric transition matrix
    res = row_transform(res)
    res = column_transform(res)

    return res

In [2]:
S = lambda x: [0xC, 5, 6, 0xB, 9, 0, 0xA, 0xD, 3, 0xE, 0xF, 8, 4, 7, 1, 2][x] # PRESENT SBox
UTM = UT_matrix(S, 4, 4)
print(UTM) # Note that the 2-adic absolute values of the elements are either 0, 1/4, 1/2 or 1

[[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  1 -2 -1  2  1 -2  0  0 -2  4  2 -4]
 [ 0  0  1  0  0  0  0 -1  1  0 -1 -1 -1  1  0  2]
 [ 0  0  0  1  0  0  0 -1  1 -1  0 -1 -1  2  0  0]
 [ 1  0  0 -1 -1  0  0  2 -1  1  1 -1  2 -1 -2  0]
 [ 0  1  0 -1  0 -1  0  2  0 -1  1  0  0  2 -1 -2]
 [ 0  0  1 -1  0  0 -1  1  0  1  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1 -1  0  1 -1  0]
 [ 1 -1 -1  2  0  0  1 -1 -1  2  2 -3  0 -1 -2  2]
 [ 0  0  0  1  1 -1 -1  1  0  0  1 -2 -1  1  0  0]
 [ 0  0  0  1  0  0  1 -2  0  1  1 -3  0 -1 -2  4]
 [ 0  0  0  1  0  0  0 -1  0  0  1 -2  0  0 -1  2]
 [ 1 -1 -1  1 -1  1  1  0 -1  2  2 -3  1 -2 -2  2]
 [ 0  0  0  0  0  0  0  1  0  0  1 -1  0  0 -1  0]
 [ 0  0  0  0  0  0  0  0  0  1  1 -2  0 -1 -1  2]
 [ 0  0  0  0  0  0  0  0  0  0  1 -1  0  0 -1  1]]


In [3]:
from math import log2
# We collect all non-zero correlation transition and -log2 of their corresponding 2-adic absolute value
sbox_ultrametric_transition_conditions = []
for i in range(16):
    for j in range(16):
        c = UTM[i, j]
        if c != 0:
            sbox_ultrametric_transition_conditions.append(tuple((j>>k) & 1 for k in range(4)) + tuple((i>>k) & 1 for k in range(4)) + (int(round(log2(int(abs(c)) & (-int(abs(c)))))), ))

In [4]:
## 2 Key addition
# from example 4.3 follows that the key addition can be modeled as $u \preccurlyeq v$
def model_key_addition(model, u, v):
    for i in range(64):
        model.Add(u[i] <= v[i])
    return model

In [5]:
## 3 Model the round function
# We use the AddAllowedAssignments constraint to non-zero correlation sbox transitions, additionally we apply theorem 4.5.2 for the bit permutation and we track -log2 of the 2-adic absolute value of the transition, to use for the bound later.
P = [0, 16, 32, 48, 1, 17, 33, 49, 2, 18, 34, 50, 3, 19, 35, 51, 4, 20, 36, 52, 5, 21, 37, 53, 6, 22, 38, 54, 7, 23, 39, 55, 8, 24, 40, 56, 9, 25, 41, 57, 10, 26, 42, 58, 11, 27, 43, 59, 12, 28, 44, 60, 13, 29, 45, 61, 14, 30, 46, 62, 15, 31, 47, 63]
def model_round_function(model, u, v):
    # variables that track the correlation
    cor_vars = [model.NewIntVar(0, 2, "") for _ in range(16)]

    # for each sbox apply constraints
    for i in range(16):
        uu = u[4*i: 4*i+4]
        vv = [v[P[j]] for j in range(4*i, 4*i+4)]
        model.AddAllowedAssignments(uu + vv + cor_vars[i:i+1], sbox_ultrametric_transition_conditions)

    # return models and correlation variables for later use
    return model, cor_vars

In [6]:
# we apply example 4.1 to model the input set $u \and \mathbb{F}_2^n$ in the ultrametric integral basis
def model_input_set(model, u, v):
    cor_vars = [model.NewBoolVar("") for _ in range(64)]
    for i in range(64):
        model.Add(v[i] <= u[i])
        model.Add(cor_vars[i] == u[i] - v[i])
    return model, cor_vars

In [7]:
## 5 build a complete model
from ortools.sat.python import cp_model

# input and output exponents
u = [1, 0, 0, 0]*4 + [0]*48 # we skip the first round, as the input set saturates an sbox of the first layer.
v = [0, 1] + [0]*62

# The model
model = cp_model.CpModel()

# We track input and output exponents before and after the key addition and round function
exponents = [[model.NewBoolVar("") for _ in range(64)] for _ in range(7)]
# We also track all variables that influence the correlation of the trail
cor_vars = []

# input set constraints
model, ecv = model_input_set(model, u, exponents[0])
cor_vars += ecv

# output function condition
for i in range(64):
    model.Add(exponents[-1][i] == v[i])

# model rounds
for r in range(3):
    model = model_key_addition(model, *exponents[2*r:2*(r+1)])
    model, ecv = model_round_function(model,  *exponents[2*r+1:2*(r+1)+1])
    cor_vars += ecv

# get maximum correlation over all trails
model.Minimize(sum(cor_vars))

# setup solver
solver = cp_model.CpSolver()
status = solver.Solve(model)
# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL:
    print(f"All trails have 2-adic absolute value <= {2**-solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print("All trails have 2-adic absolute value 0")
else:
    print("error")

All trails have 2-adic absolute value <= 0.25


In conclusion, we can validate the result from section 5.2 programmatically.